In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.getenv('SAM_PATH'))
from samutil import MultiThreading
from tqdm.notebook import tqdm
import json

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = '03ba24122703f9cff69789b9ebbd342e'

In [44]:
page_results = []
years_check = [i for i in range(1980,2022)]

In [45]:
def get_results(years_check):
    for i in tqdm(years_check):
        api = tmdb.Discover()
        total_pages_to_loop = api.movie(year=i,page=1)['total_pages']
        for loop in tqdm(range(1,total_pages_to_loop)):
            movie = api.movie(year=i,page=loop)
            page_results.append(movie['results'])
    return

In [47]:
mt = MultiThreading(10,years_check,page_results)

In [ ]:
mt.Run(get_results)

In [49]:
def parse_discover_results(page_results):
    cleaned_results = []
    
    for result in range(len(page_results)):
        try:
            tmdb_id = page_results[result]['id']
            title = page_results[result]['title']
            overview = page_results[result]['overview']
            popularity = page_results[result]['popularity']
            release = page_results[result]['release_date']
            vote_average = page_results[result]['vote_average']
            cleaned_results.append((tmdb_id,title,overview,popularity,release,vote_average))
        except:
            pass
        
    return cleaned_results

In [56]:
all_results = []
for page in range(len(page_results)):
    all_results.append(pd.DataFrame(parse_discover_results(page_results[page])))

In [57]:
df = pd.concat(all_results)

In [59]:
df = df.drop_duplicates()

In [181]:
df.columns = ['tmdb_id','movie','overview','popularity','release_date','vote_average']
df = df[df.popularity >= 5]

In [182]:
df.to_csv('tmdb_discover_results.csv',index=False)

In [9]:
df = pd.read_csv('tmdb_discover_results.csv',lineterminator='\n')

# Get more field---


In [11]:
def get_response(search_query):
    search = tmdb.Search()
    response = search.movie(query=search_query)
    return response

In [12]:
def get_film_id(search_query):
    search = tmdb.Search()
    response = search.movie(query=search_query)
    return response['results'][0]['id']

In [23]:
def get_film_response(film_id):
    film_response = tmdb.Movies(film_id)
    return film_response

In [7]:
def build_film_dic(film_id,film_response):
    film_dic = {}
    
    try:
        film_dic['tmdb_id'] = film_id
    except:
        film_dic['tmdb_id'] = np.nan
    
    try:
        film_dic['budget'] = film_response.info()['budget']   
    except:
        film_dic['budget'] = np.nan
    
    try:    
        film_dic['revenue'] = film_response.info()['revenue']
    except:  
        film_dic['revenue'] = np.nan
    
    try:
        film_dic['genres'] = film_response.info()['genres']
    except:    
        film_dic['genres'] = np.nan
    
    try:
        if len(film_response.reviews()['results']) <1:
            film_dic['reviews'] = np.nan
        elif len(film_response.reviews()['results']) >0:
            film_dic['reviews'] = film_response.reviews()['results']
    except:    
        film_dic['reviews'] = np.nan
    
    try:
        if len(film_response.keywords()['keywords']) <1:
            film_dic['keywords'] = np.nan
        elif len(film_response.keywords()['keywords']) >1:
            film_dic['keywords'] = film_response.keywords()['keywords']
    except:    
        film_dic['keywords'] = np.nan
    
    try:
        film_dic['production_companies'] = film_response.info()['production_companies']
    except:    
        film_dic['production_companies'] = np.nan
    
    try:
        film_dic['production_countries'] = film_response.info()['production_countries']
    except:    
        film_dic['production_countries'] = np.nan
        
    try:
        film_dic['cast'] = film_response.credits()['cast'] 
        
    except:
        film_dic['cast'] = np.nan
    
    try:
        film_dic['crew'] = film_response.credits()['crew'] 
    except:
        film_dic['crew'] = np.nan
    
    
    return film_dic
    
    return film_dic

In [12]:
results = []

In [13]:
def get_film_details(film_ids):
    
    for film_id in tqdm(film_ids):
      

        film_response = get_film_response(film_id)
        film_dic = build_film_dic(film_id,film_response)
        results_new.append(film_dic)

            
    return results

In [14]:
mt = MultiThreading(20,df.tmdb_id.to_list(),output=results)

In [ ]:
mt.Run(get_film_details)

# Restructuring Cast and Crew
---

In [10]:
import tmdbsimple as tmdb
tmdb.API_KEY = '03ba24122703f9cff69789b9ebbd342e'

In [121]:
all_results = []

In [122]:
def get_cast_crew(film_ids):
    for id_ in tqdm(film_ids):
        try:
            film_response = get_film_response(id_)
            cast_dic_list = film_response.credits()['crew']
            for dic in cast_dic_list:
                
                if dic['popularity'] >= 2:
                    dic['tmdb_id'] = id_
                    all_results.append(dic)
                else:
                    pass
        except:
            pass
    return

In [123]:
mt = MultiThreading(20,df.tmdb_id.to_list(),all_results)

In [124]:
mt.Run(get_cast_crew)

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

...

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [125]:
import json

with open('crew_2.jsonl', 'w') as outfile:
    for entry in all_results:
        json.dump(entry, outfile)
        outfile.write('\n')

In [126]:
%%time
with open('crew_2.jsonl') as f:
    lines = f.read().splitlines()
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']
pd.json_normalize(df_inter['json_element'].swifter.apply(json.loads))

Pandas Apply:   0%|          | 0/15959 [00:00<?, ?it/s]

CPU times: user 558 ms, sys: 12.6 ms, total: 571 ms
Wall time: 569 ms


,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,tmdb_id
0,False,2,82702,Acting,Michael Jackson,Michael Jackson,3.796,/oc5FWwO5VYBjkKkIHTxoCtFXJ1G.jpg,52fe48b39251416c750b0819,Production,Executive Producer,90724
1,False,2,82702,Acting,Michael Jackson,Michael Jackson,3.796,/oc5FWwO5VYBjkKkIHTxoCtFXJ1G.jpg,5ef1013b9a8a8a0035ae8d1f,Crew,Choreographer,90724
2,False,2,16863,Directing,Todd Haynes,Todd Haynes,2.199,/wBDazO2j49uoquoHTEDxcniyGGC.jpg,5691cbc1c3a3684cff000831,Production,Executive Producer,340487
3,False,1,1017377,Production,Stephanie Holbrook,Stephanie Holbrook,2.514,None,546df8de9251412baf001e4f,Production,Casting,300654
4,False,2,32608,Production,Brad Simpson,Brad Simpson,2.580,None,5c8ec4ba9251410fe7a26f2b,Production,Producer,472674
...,...,...,...,...,...,...,...,...,...,...,...,...
15954,False,1,1113,Production,Lucinda Syson,Lucinda Syson,2.852,None,52fe479f9251416c7509ff1b,Production,Casting,18823
15955,False,2,18865,Directing,Louis Leterrier,Louis Leterrier,2.633,/pJl1hNtZOIxrdzQjwGlicWWGSXg.jpg,52fe479e9251416c7509fe4b,Directing,Director,18823
15956,False,2,578,Directing,Ridley Scott,Ridley Scott,3.878,/zABJmN9opmqD4orWl3KSdCaSo7Q.jpg,52fe423bc3a36847f800e117,Directing,Director,348
15957,False,2,239,Writing,Jon Vitti,Jon Vitti,2.804,/yoljNk80dUPs5sftsPHiDf31kQ9.jpg,52fe4468c3a368484e0215fd,Writing,Writer,23398


In [78]:
df_inter

,json_element
0,"""adult"""
1,"""gender"""
2,"""id"""
3,"""known_for_department"""
4,"""name"""
...,...
280756,"""cast_id"""
280757,"""character"""
280758,"""credit_id"""
280759,"""order"""


# Get Tag and Movie Poster
--

In [55]:
def get_film_response(film_id):
    film_response = tmdb.Movies(film_id)
    return film_response

In [56]:
def build_film_dic(film_id,film_response):
    film_dic = {}
    
    try:
        film_dic['tmdb_id'] = film_id
    except:
        film_dic['tmdb_id'] = np.nan
        
    try:
        film_dic['poster_path'] = 'https://image.tmdb.org/t/p/w500'+film_response.info()['poster_path']  
    except:
        film_dic['poster_path'] = np.nan
    
    try:
        film_dic['tagline'] = film_response.info()['tagline']   
    except:
        film_dic['tagline'] = np.nan
    
   
    
    return film_dic
    


In [57]:
all_results = []

In [58]:
def get_film_details(film_ids):
    
    for film_id in tqdm(film_ids):
      

        film_response = get_film_response(film_id)
        film_dic = build_film_dic(film_id,film_response)
        all_results.append(film_dic)

            
    return results

In [59]:
x = pd.read_json('/Users/sam.ho/Documents/sam_personal/streamlit_apps/imdb_network_analysis/data/main/tmdb_data_main.json')

In [60]:
x['tmdb_id'].to_list()

[694,
 578,
 238,
 288985,
 185,
 4488,
 8393,
 11449,
 123,
 1891,
 1367,
 8689,
 5336,
 5689,
 948,
 9659,
 28,
 11542,
 335,
 923,
 447332,
 138843,
 299536,
 335983,
 505262,
 671,
 390043,
 284053,
 345940,
 354912,
 400155,
 346910,
 333339,
 518764,
 398818,
 8587,
 439079,
 324786,
 433945,
 383498,
 672,
 425,
 808,
 9023,
 585,
 9820,
 39100,
 9637,
 24752,
 597,
 120,
 39105,
 121,
 557,
 11544,
 39103,
 9487,
 7342,
 10144,
 39101,
 268,
 149,
 11587,
 11186,
 106,
 19185,
 39323,
 8392,
 1368,
 8913,
 377,
 218,
 562,
 114,
 10014,
 162,
 11674,
 10674,
 11970,
 9732,
 2300,
 11932,
 18937,
 13761,
 9444,
 14444,
 637,
 18491,
 317442,
 24428,
 198663,
 106646,
 122917,
 260346,
 157336,
 177572,
 109445,
 9552,
 9948,
 102651,
 250546,
 265712,
 76338,
 270946,
 45243,
 238636,
 34584,
 1369,
 9730,
 10925,
 105,
 87,
 1374,
 85,
 9340,
 620,
 9731,
 44251,
 34433,
 39106,
 39104,
 9479,
 39102,
 278,
 888,
 11811,
 10112,
 1634,
 680,
 13,
 674,
 42246,
 12429,
 862,
 14

In [61]:
mt = MultiThreading(20,x['tmdb_id'].to_list(),all_results)

In [62]:
mt.Run(get_film_details)

  0%|          | 0/984 [00:00<?, ?it/s]

  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 0/984 [00:00<?, ?it/s]

  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 0/983 [00:00<?, ?it/s]

...

  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 0/983 [00:00<?, ?it/s]

Exception in thread Thread-45:
Traceback (most recent call last):
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-58-3484b8e527ab>", line 11, in get_film_details
    return results
NameError: name 'results' is not defined

Exception in thread Thread-49:
Traceback (most recent call last):
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-58-3484b8e527ab>", line 11, in get_film_details
    return results
NameError: name 'results' is not defined

Exception in thread Thread-46:
Traceback (most recent call last):
  File "/U

Exception in thread Thread-47:
Traceback (most recent call last):
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-58-3484b8e527ab>", line 11, in get_film_details
    return results
NameError: name 'results' is not defined

Exception in thread Thread-64:
Traceback (most recent call last):
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/sam.ho/opt/anaconda3/envs/sam_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-58-3484b8e527ab>", line 11, in get_film_details
    return results
NameError: name 'results' is not defined



In [66]:
y = pd.DataFrame(all_results)

In [68]:
df_use = pd.merge(x,y,how='left')

In [73]:
df_use.to_json('/Users/sam.ho/Documents/sam_personal/streamlit_apps/imdb_network_analysis/data/main/tmdb_data_main.json')